# CuPy について
* CuPy は、外部インタフェースが NumPy 互換で、内部的に GPU (Cuda) を使っている。
* macOS Sierra 以降では DYLD_LIBRARY_PATH の上書きができなくなったので、Cuda 関連のライブラリを ~/lib などにシンボリックリンクを貼る必要がある
    * /Developer/NVIDIA/CUDA-9.1/lib/ 以下のファイルのシンボリックリンクを ~/lib につくる  
    * 他にも $(HOME)/lib:/usr/local/lib/:/lib/:/usr/lib あたりにシンボリックリンクを作っても良い

In [2]:
import cupy as cp
import numpy as np
a = cp.array([1,2,3,4,5])
b = cp.array([6,7,8,9,10])
c = a + b
print (c)

[ 7  9 11 13 15]


In [3]:
d = cp.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
e = cp.array([
    [9,8,7],
    [6,5,4],
    [3,2,1]
])
f = cp.dot(d,e)
print(f)

[[ 30  24  18]
 [ 84  69  54]
 [138 114  90]]


# Python の list に格納した cupy.ndarray 同士で計算を行う

In [4]:
args = []
rets = []

args.append(cp.array([10,20,30,40,50]))
args.append(cp.array([1,2,3,4,5]))
args.append(cp.array([2,3,4,5,6]))
args.append(cp.array([3,4,5,6,7]))
args.append(cp.array([4,5,6,7,8]))

rets.append(args[0] + args[0])
rets.append(args[0] + args[1])
rets.append(args[0] + args[2])
rets.append(args[0] + args[3])
rets.append(args[0] + args[4])

print(rets[0].get())
print(rets[1].get())
print(rets[2].get())
print(rets[3].get())
print(rets[4].get())


[ 20  40  60  80 100]
[11 22 33 44 55]
[12 23 34 45 56]
[13 24 35 46 57]
[14 25 36 47 58]


# numpy, cupy 両対応のコード

In [3]:
def calc(x, y):
    xp = cp.get_array_module(x)
    return xp.dot(x, y)


x_cpu = np.array([[1,2],[3,4]])
y_cpu = np.array([[5,6],[7,8]])
print(calc(x_cpu, y_cpu))

x_gpu = cp.asanyarray(x_cpu, dtype='float32')
y_gpu = cp.asanyarray(y_cpu, dtype='float32')
print(calc(x_gpu, y_gpu).get())


[[19 22]
 [43 50]]


[[19. 22.]
 [43. 50.]]


# Numpy 向けに作ったクラスを Cupy に対応させる
* やりたいこと
    * Cupy がないマシンでは Numpy で動かしたい。ただし、動作時に自動判別をする必要はない。
    * ベースのクラスを Numpy 向けに作る
    * それを継承したクラスをデコレーターで Cupy 対応させる
* まずは Numpy ベースのクラスを作る

In [7]:
class BaseClass:
    def __init__(self, x):
        self.store = x
        
    def add(self, x, xp = np):
        print("add's xp is {}".format(xp))
        self.store = xp.add(self.store, x)

    def sub(self, x, xp = np):
        print("sub's xp is {}".format(xp))
        self.store = xp.subtract(self.store, x)


cpu = BaseClass(np.array([1,2,3]))
cpu.add(np.array([0,0,1]))
cpu.sub(np.array([1,0,0]))
print(cpu.store)

add's xp is <module 'numpy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/numpy/__init__.py'>
sub's xp is <module 'numpy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/numpy/__init__.py'>
[0 2 4]


* Cupy 対応の decorator を BaseClass に適用する

In [9]:
import functools    


def cupy_decorator(func):
    @functools.wraps(func)
    def __func(self, x):
        return func(self, x, xp=cp.get_array_module(x))
    return __func


def GPU(clazz):
    for property_name in dir(clazz):
        if property_name in ['add', 'sub']:
            method = getattr(clazz, property_name)
            setattr(clazz, property_name, cupy_decorator(method))
    return clazz


@GPU
class GPUClass(BaseClass):
    pass

# 引数が numpy.ndarray の場合は numpy が使われる
cpu = GPUClass(np.array([1,2,3]))
cpu.add(np.array([0,0,1]))
cpu.sub(np.array([1,0,0]))
print(cpu.store)

# 引数が cupy.ndarray の場合は cupy が使われる
gpu = GPUClass(cp.array([1,2,3]))
gpu.add(cp.array([0,0,1]))
gpu.sub(cp.array([1,0,0]))
print(gpu.store.get())

add's xp is <module 'numpy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/numpy/__init__.py'>
sub's xp is <module 'numpy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/numpy/__init__.py'>
[0 2 4]
add's xp is <module 'cupy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/cupy/__init__.py'>
sub's xp is <module 'cupy' from '/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/cupy/__init__.py'>
[0 2 4]
